# Upload to data.gouv

Dans cette partie, on envoie les données sur le site data.gouv.fr

In [107]:
import geopandas as gpd
import pandas as pd
import requests
from bs4 import BeautifulSoup
import zipfile
import py7zr
import os
import shutil
import getpass
import glob
import json

In [2]:
http_proxy = getpass.getpass(prompt='http_proxy')
https_proxy = getpass.getpass(prompt='http_proxy')
proxies = { "http"  : http_proxy, "https" : https_proxy}

http_proxy ········
http_proxy ········


In [84]:
API_KEY = getpass.getpass(prompt='API_KEY')

API_KEY ········


In [86]:
API = 'https://www.data.gouv.fr/api/1'
DATASET = '670e7c115a3cc3e558e87afa'
HEADERS = {'X-API-KEY': API_KEY}

In [7]:
liste_termine = glob.glob("files_ocsge/*")

In [ ]:
for file in liste_termine[100:]:
    print(file)
    url = API + '/datasets/{}/upload/'.format(DATASET)
    response = requests.post(url, files={'file': open(file, 'rb'),}, headers=HEADERS, proxies=proxies)

Liste des fichiers dispo sur data.gouv

In [100]:
url = 'https://www.data.gouv.fr/fr/datasets/occupation-du-sol-a-grande-echelle-ign-et-cerema/'

In [125]:
page = requests.get(url, proxies=proxies)
soup = BeautifulSoup(page.text, 'html.parser')
script = soup.find('script').text
ressources = json.loads(script)['distribution']

In [136]:
ressources_link = [{'id' : ressource['@id'], 'name' : ressource['name'], 'url' : ressource['contentUrl']} for ressource in ressources]

In [138]:
ressources_link = pd.DataFrame(ressources_link)

In [150]:
temp = (ressources_link.name.str.split('shp', expand=True)[[1]])[1].str.split('-', expand=True)

In [156]:
ressources_link['proj'] = temp[[1]]
ressources_link['dep'] = temp[[2]]
ressources_link['year'] = temp[[3]]

In [157]:
ressources_link.head()

,id,name,url,proj,dep,year
0,ca4b0b93-0166-44ef-8ff7-eba3245db99a,ocs-ge-2-0-shp-rgaf09utm20-d972-2022-01-01.par...,https://static.data.gouv.fr/resources/occupati...,rgaf09utm20,d972,2022
1,42a953b5-14c0-4f69-b9f9-39e291ef4ff5,ocs-ge-2-0-shp-rgaf09utm20-d972-2017-01-01.par...,https://static.data.gouv.fr/resources/occupati...,rgaf09utm20,d972,2017
2,4583734b-1a2d-4630-8975-7e2c497befaa,ocs-ge-2-0-shp-lamb93-d095-2021-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,d095,2021
3,3cc771a8-dfa5-4fb3-9cf1-f1c6f7b0576b,ocs-ge-2-0-shp-lamb93-d095-2018-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,d095,2018
4,d79d16d0-bd15-4a04-8e25-6abcac552da2,ocs-ge-2-0-shp-lamb93-d094-2021-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,d094,2021


In [174]:
ressources_link.to_json('ressources_link.json', orient='records')

In [170]:
print(ressources_link.to_markdown(index=False))

| id                                   | name                                               | url                                                                                                                                                       | proj        | dep   |   year |
|:-------------------------------------|:---------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------|:------------|:------|-------:|
| ca4b0b93-0166-44ef-8ff7-eba3245db99a | ocs-ge-2-0-shp-rgaf09utm20-d972-2022-01-01.parquet | https://static.data.gouv.fr/resources/occupation-du-sol-a-grande-echelle-ign-et-cerema/20250121-171436/ocs-ge-2-0-shp-rgaf09utm20-d972-2022-01-01.parquet | rgaf09utm20 | d972  |   2022 |
| 42a953b5-14c0-4f69-b9f9-39e291ef4ff5 | ocs-ge-2-0-shp-rgaf09utm20-d972-2017-01-01.parquet | https://static.data.gouv.fr/resources/occupation-du-sol-a-grand